# **Introduction**

We know a high ability of CNN through this project.
So, we would like to know a effection of transfer-learning on CNN.

In this time, we will address 2 classification problem, which is to distingish dog or cat, with pre-trained vgg 16 model. The model is pre-trained with ILSVRC2012 dataset (class: 1000, datasize:135 milions).

*this programming environment is kaggle's notebook*

In [ ]:
import glob
import os.path
import random
from tqdm import tqdm 
import pandas as pd
import numpy as np
import json
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

torch.manual_seed(123)
np.random.seed(223)
random.seed(234)

In [ ]:
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
import zipfile

with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    df = z.extractall(".")

In [ ]:
# 前処理クラスを作成
class BaseTransform():
    
    def __init__(self, resize, mean, std):
            self.base_transform = {
            'train': transforms.Compose([
                transforms.Resize(resize),
                transforms.RandomResizedCrop(resize, scale=(0.5, 1.0)),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize(mean, std)
                ]),
            'val': transforms.Compose([
                    transforms.Resize(resize),
                    transforms.CenterCrop(resize),
                    transforms.ToTensor(),
                    transforms.Normalize(mean, std)
                ])
            }
        
    def __call__(self, img, phase='train'):
        return self.base_transform[phase](img)
    
img_sample = '../working/train/dog.8412.jpg'
img = Image.open(img_sample)

plt.imshow(img)
plt.show()
    
resize = 224
mean = (0.485, 0.456, 0.406) # 転移学習モデル学習データに使われた平均と標準偏差で前処理
std = (0.229, 0.224, 0.225)

transform = BaseTransform(resize, mean, std) # ([色,高さ,幅])
img_transformed = transform(img)

img_transformed = img_transformed.numpy().transpose((1,2,0)) # ([高さ,幅,色])
img_transformed = np.clip(img_transformed, 0, 1) # 最小値、最大値を0,1に
plt.imshow(img_transformed)
plt.show()

In [ ]:
# データのセットを作る---------------------------------

IMG_FILE = '../working/train'

filenames = os.listdir(IMG_FILE)
labels = [x.split(".")[0] for x in filenames]
df = pd.DataFrame({"filename": filenames, "label": labels})
print(df.label.value_counts())

In [ ]:
# テストデータを作る
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.1, random_state = 1, stratify = df.label)
train.sample(frac=1, random_state=1)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

In [ ]:
# バリーデーションを作成
train, val = train_test_split(train, test_size=0.2, random_state = 2, stratify = train.label)
train.sample(frac=1, random_state=1)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)

In [ ]:
rootpath = '../working/train/'

def make_path_list(df):
    
    dogs_path = []
    cats_path = []
    
    for i in list(df[df.label=="dog"].filename):
        dogs_path.append(os.path.join(rootpath + i))
        
    for i in list(df[df.label=="cat"].filename):
        cats_path.append(os.path.join(rootpath + i))
    
    #dog_data = np.concatenate([np.array(dogs_path).reshape(-1, 1), np.full(len(dogs_path), 0).reshape(-1, 1)], axis=1)
    #cat_data = np.concatenate([np.array(cats_path).reshape(-1, 1), np.full(len(cats_path), 1).reshape(-1, 1)], axis=1)
    
    return np.concatenate([dogs_path, cats_path], axis=0)

train_path = make_path_list(train)
val_path = make_path_list(val)
test_path = make_path_list(test)

print(train_path)
print(val_path)

# 一次元同士の結合は np.stack() で可能

In [ ]:
train_path[10000].split('/')[3][0:3]

In [ ]:
class Dataset(data.Dataset):
    
    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list
        self.transform = transform
        self.phase = phase
        
    def __len__(self):
        return len(self.file_list)
    
    def __getitem__(self, index):
        
        img_path = self.file_list[index]
        img = Image.open(img_path)
        
        img_transformed = self.transform(
        img, self.phase)
        
        if self.phase == 'train':
            label = img_path.split('/')[3][0:3]
        elif self.phase == 'val':
            label = img_path.split('/')[3][0:3]
        
        # このような形式でラベル付けしなければ学習できない
        
        if label == 'dog':
            label = 0
        elif label == 'cat':
            label = 1
            
        return img_transformed, label
    
# len, getitem は宣言しなければならない
    
train_dataset = Dataset(file_list=train_path, transform=BaseTransform(resize, mean, std), phase='train')
val_dataset = Dataset(file_list=val_path, transform=BaseTransform(resize, mean, std), phase='val')

In [ ]:
index = 0
print(train_dataset.__getitem__(index)[0].size())

In [ ]:
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

dataloaders_dict = {'train': train_dataloader, 'val':val_dataloader}


batch_iterator = iter(dataloaders_dict['train'])
#for _, j in batch_iterator:
#    print(i)
#    print(j)

In [ ]:
# 転移学習----------------------------------------------------------------------
net = models.vgg16(pretrained=True)

# add a layer to last layer
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

net.train()

In [ ]:
cost = nn.CrossEntropyLoss()

In [ ]:
params_to_update = []

update_params_names = ['classifier.6.weight', 'classifier.6.bias']

# 転移学習用のパラメータは変化しないように設定
for name, param in net.named_parameters():
    if name in update_params_names:
        param.requires_grad = True
        params_to_update.append(param)
        print(name)
    else:
        param.requires_grad = False

In [ ]:
optimizer = optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

In [ ]:
# テスト
test_dataset = Dataset(file_list=val_path, transform=BaseTransform(resize, mean, std), phase='val')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
def train_model(net, dataloaders_dict, cost, optimizer, num_epochs, test_dataloader):
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    net.to(device)
    
    torch.backends.cudnn.benchmark = True
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの学習と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):
                
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = cost(outputs, labels)
                    _, preds = torch.max(outputs, 1)  
                    
  
                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)  
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
    epoch_loss = 0.0  # testepochの損失和
    epoch_corrects = 0  # testepochの正解数

    for inputs_t, labels_t in tqdm(test_dataloader):
        
        inputs_t = inputs_t.to(device)
        labels_t = labels_t.to(device)
        
        outputs_t = net(inputs_t)
        loss_t = cost(outputs_t, labels_t)  
        _, preds_t = torch.max(outputs_t, 1)

        epoch_loss += loss_t.item() * inputs.size(0)  

        epoch_corrects += torch.sum(preds_t == labels_t.data)


    epoch_loss = epoch_loss / len(test_dataloader.dataset)
    epoch_acc = epoch_corrects.double() / len(test_dataloader.dataset)

    print('Test Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [ ]:
num_epochs = 2
train_model(net, dataloaders_dict, cost, optimizer, num_epochs=num_epochs, test_dataloader=test_dataloader)

# We can obtain very good result despite of adding only a layer!

In [ ]:
# ファインチューニング
net_2 = models.vgg16(pretrained=True)

net_2.classifier[6] = nn.Linear(in_features=4096, out_features=2)
net_2.train()

In [ ]:
cost = nn.CrossEntropyLoss()

In [ ]:
params_to_update1= []
params_to_update2 = []
params_to_update3 = []

update_params_names1 = ['features']
update_params_names2 = ['classifier.0.weight', 'classifier.0.bias', 'classifier.3.weight', 'classifier.3.bias']
update_params_names3 = ['classifier.6.weight', 'classifier.6.bias']

for name, param in net_2.named_parameters():
    if update_params_names1[0] in name:
        param.requires_grad = True
        params_to_update1.append(param)
        print('1: {}'.format(name))
        
    elif name in update_params_names2:
        param.requires_grad = True
        params_to_update2.append(param)
        print('2: {}'.format(name))
        
    elif name in update_params_names3:
        param.requires_grad = True
        params_to_update3.append(param)
        print('3: {}'.format(name))
        
    else:
        params.requires_grad = False
        print('Nothing: {}'.format(name))
        
optimizer_t = optim.SGD([
    {'params': params_to_update1, 'lr': 0.001},
    {'params': params_to_update2, 'lr': 0.001},
    {'params': params_to_update3, 'lr': 0.001}
    ],
    momentum = 0.9)

In [ ]:
def train_model_fine(net, dataloaders_dict, cost, optimizer, num_epochs, test_dataloader):
    
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    
    net.to(device)
    
    torch.backends.cudnn.benchmark = True
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('----------------')
        
        for phase in ['train', 'val']:
            
            if phase in 'train':
                net.train()
            else:
                net.eval()
                
            epoch_loss = 0.0
            epoch_corrects = 0

            if (epoch == 0) and (phase == 'train'):
                continue

            for inputs, labels in tqdm(dataloaders_dict[phase]):

                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    outputs = net(inputs)
                    loss = cost(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    epoch_loss += loss.item() * inputs.size(0)
                    epoch_corrects += torch.sum(preds == labels.data)


            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double() / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
        
        
    epoch_loss = 0.0  # testepochの損失和
    epoch_corrects = 0  # testepochの正解数

    for inputs_t, labels_t in tqdm(test_dataloader):
        
        inputs_t = inputs_t.to(device)
        labels_t = labels_t.to(device)
        
        outputs_t = net(inputs_t)
        loss_t = cost(outputs_t, labels_t)  
        _, preds_t = torch.max(outputs_t, 1)

        epoch_loss += loss_t.item() * inputs.size(0)  

        epoch_corrects += torch.sum(preds_t == labels_t.data)


    epoch_loss = epoch_loss / len(test_dataloader.dataset)
    epoch_acc = epoch_corrects.double() / len(test_dataloader.dataset)

    print('Test: Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

In [ ]:
num_epochs = 2
train_model_fine(net_2, dataloaders_dict, cost, optimizer_t, num_epochs=num_epochs, test_dataloader=test_dataloader)

# Conclusion

We are surprised by this result Because training epoch is just 2 and accuracy is very high!
This represent a high effection with transfer-learning.
But, we consider that good result can't always be obtained in any probrem with using transfer-learning.
Because pre-trained model has high probability for overffiting and not fitting some tasks because of difference of learning content.
So, We will use a pre-trained model with paying atention if effective or not.